## Generative Latent Replay with GMMs

Code to test latent replay on benchmark problems and compare with proposed generative latent replay strategies i.e. normalising bottleneck representations and sampling from fitted GMM on latent space.

In [71]:
import importlib

import torch
from torch.nn import CrossEntropyLoss
from torch.optim import SGD

from avalanche.benchmarks.classic import PermutedMNIST #,PermutedOmniglot, RotatedOmniglot
from avalanche.models import SimpleMLP
from avalanche.training import Naive, AR1
import models

from torchvision import transforms

importlib.reload(models)

<module 'models' from 'c:\\Users\\jacob\\Documents\\GitHub\\generative-latent-replay\\models\\__init__.py'>

In [72]:
# Config
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# model
model = SimpleMLP(num_classes=10)

# CL Benchmark Creation
# Original AR1* usese CORE50 (n,3,128,128) and pretrained mobilenet
transform = transforms.Compose([
    #transforms.Lambda(lambda x: x.convert('RGB')),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1))
])

perm_mnist = PermutedMNIST(n_experiences=3, train_transform=transform, eval_transform=transform)
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

# Prepare for training & testing
criterion = CrossEntropyLoss()

# Continual learning strategy
cl_strategy = models.AR1(criterion, train_mb_size=32, train_epochs=2, eval_mb_size=32, device=device) #LatentReplay(criterion, train_mb_size=32, train_epochs=2, eval_mb_size=32, device=device)

# train and test loop over the stream of experiences
results = []
for train_exp in train_stream:
    cl_strategy.train(train_exp)
    results.append(cl_strategy.eval(test_stream))

-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --


c:\Users\jacob\Documents\GitHub\generative-latent-replay\models\ar1.py:105: UserWarning: The AR1 strategy implementation is in an alpha stage and is not perfectly aligned with the paper implementation. Please use at your own risk!
  warnings.warn(


 27%|██▋       | 508/1875 [02:59<08:23,  2.72it/s]   

In [4]:
# Config
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# model
model = SimpleMLP(num_classes=10)

# CL Benchmark Creation
perm_mnist = PermutedMNIST(n_experiences=3)
train_stream = perm_mnist.train_stream
test_stream = perm_mnist.test_stream

# Prepare for training & testing
optimizer = SGD(model.parameters(), lr=0.001, momentum=0.9)
criterion = CrossEntropyLoss()

# Continual learning strategy
cl_strategy = Naive(
    model, optimizer, criterion, train_mb_size=32, train_epochs=2,
    eval_mb_size=32, device=device)

# train and test loop over the stream of experiences
results = []
for train_exp in train_stream:
    cl_strategy.train(train_exp)
    results.append(cl_strategy.eval(test_stream))


-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|██████████| 1875/1875 [3:31:30<00:00,  6.77s/it]     
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.4868
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8600
100%|██████████| 1875/1875 [00:31<00:00, 59.39it/s]
Epoch 1 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.2590
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9243
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████| 313/313 [00:04<00:00, 69.66it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1826
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9465
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████| 313/313 [00:04<00:00, 74.73it/s]
> Eval on experience 1 (Task 1) from test stream ended.
	Loss_Exp/ev